In [2]:
# ECS289G_Term_Project/adversarial_model.ipynb

import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, OneHotEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from imblearn.over_sampling import SMOTE, RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

In [3]:
########################################
# Step 1: Load and Preprocess Data
########################################

path = "/Users/harshil/Developer/GitHub_Repos/ECS_289G/data/processsed/transformed/transformed_dataset-yrs-23.csv"
df = pd.read_csv(path, low_memory=False)

# Map action_taken to binary
loan_approved_mapping = {
    1: 1,  # Loan originated -> Approved
    2: 0,  # Approved but not accepted -> Denied
    3: 0,  # Denied
    4: 0,  # Withdrawn
    5: 0,  # Incomplete
    6: 1,  # Purchased
    7: 0,  # Preapproval denied
    8: 0   # Preapproval approved not accepted
}

df['loan_approved'] = df['action_taken'].map(loan_approved_mapping)
df_binary = df[df['loan_approved'].notnull()].copy()
df_binary['loan_approved'] = df_binary['loan_approved'].astype(int)
df_binary.drop('action_taken', axis=1, inplace=True)

# Sensitive features
sensitive_features = [
    'race_0', 'race_1', 'race_2', 'race_3', 'race_4',
    'race_5', 'race_6', 'race_7', 'race_8',
    'gender_0', 'gender_1', 'gender_2', 'gender_3',
    'ethnicity_0', 'ethnicity_1', 'ethnicity_2', 'ethnicity_3', 'ethnicity_4'
]

# Non-sensitive features for main model
feature_cols = [
    'tract_to_msa_income_percentage',
    'ffiec_msa_md_median_family_income',
    'tract_minority_population_percent',
    'interest_rate',
    'loan_type_2', 'loan_type_3', 'loan_type_4',
    'loan_purpose_2', 'loan_purpose_4', 'loan_purpose_5',
    'loan_purpose_31', 'loan_purpose_32',
    'lien_status_2',
    'construction_method_2',
    'occupancy_type_2', 'occupancy_type_3'
]

X = df_binary[feature_cols]
y = df_binary['loan_approved']
sensitive_attributes = df_binary[sensitive_features]

X_train, X_test, y_train, y_test, sensitive_train, sensitive_test = train_test_split(
    X, y, sensitive_attributes, test_size=0.2, random_state=42
)

print("Training set size:", X_train.shape[0])
print("Testing set size:", X_test.shape[0])

# Scale features
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("Feature scaling completed")

Training set size: 1281889
Testing set size: 320473
Feature scaling completed


In [4]:
########################################
# Decode Sensitive Attributes for Groups
########################################

sensitive_train_simplified = sensitive_train.copy()
sensitive_test_simplified = sensitive_test.copy()

def get_race(row):
    race_mapping = {
        'race_0': 'American Indian or Alaska Native',
        'race_1': 'Asian',
        'race_2': 'Black or African American',
        'race_3': 'Native Hawaiian or Other Pacific Islander',
        'race_4': 'White',
        # Less common aggregated into 'Other'
        'race_5': 'Other', 'race_6': 'Other', 'race_7': 'Other',
        'race_8': 'Unknown or Not Applicable'
    }
    for col in race_mapping:
        if row[col] == 1:
            return race_mapping[col]
    return 'Unknown'

def get_gender(row):
    gender_mapping = {
        'gender_0': 'Male',
        'gender_1': 'Female',
        'gender_2': 'Joint',
        'gender_3': 'Unknown or Not Applicable'
    }
    for col in gender_mapping:
        if row[col] == 1:
            return gender_mapping[col]
    return 'Unknown'

def get_ethnicity(row):
    ethnicity_mapping = {
        'ethnicity_0': 'Hispanic or Latino',
        'ethnicity_1': 'Not Hispanic or Latino',
        'ethnicity_2': 'Joint',
        'ethnicity_3': 'Other',
        'ethnicity_4': 'Unknown or Not Applicable'
    }
    for col in ethnicity_mapping:
        if row[col] == 1:
            return ethnicity_mapping[col]
    return 'Unknown'

sensitive_train_simplified['race_group'] = sensitive_train_simplified.apply(get_race, axis=1)
sensitive_train_simplified['gender_group'] = sensitive_train_simplified.apply(get_gender, axis=1)
sensitive_train_simplified['ethnicity_group'] = sensitive_train_simplified.apply(get_ethnicity, axis=1)

sensitive_test_simplified['race_group'] = sensitive_test_simplified.apply(get_race, axis=1)
sensitive_test_simplified['gender_group'] = sensitive_test_simplified.apply(get_gender, axis=1)
sensitive_test_simplified['ethnicity_group'] = sensitive_test_simplified.apply(get_ethnicity, axis=1)

# Create a demographic_group column (Race_Gender combo)
sensitive_train_simplified['demographic_group'] = sensitive_train_simplified['race_group'] + '_' + sensitive_train_simplified['gender_group']
sensitive_test_simplified['demographic_group'] = sensitive_test_simplified['race_group'] + '_' + sensitive_test_simplified['gender_group']

In [5]:
########################################
# Resampling by group to handle imbalance
########################################

X_train_combined = pd.DataFrame(X_train_scaled, columns=feature_cols)
X_train_combined['demographic_group'] = sensitive_train_simplified['demographic_group'].values
X_train_combined['loan_approved'] = y_train.values
X_train_combined = X_train_combined.reset_index(drop=True)
sensitive_train_simplified = sensitive_train_simplified.reset_index(drop=True)

assert all(X_train_combined['demographic_group'] == sensitive_train_simplified['demographic_group']), "DataFrames not aligned!"

X_resampled_list = []
y_resampled_list = []
sensitive_resampled_list = []

for group in X_train_combined['demographic_group'].unique():
    group_indices = X_train_combined[X_train_combined['demographic_group'] == group].index
    X_group = X_train_combined.loc[group_indices, feature_cols]
    y_group = X_train_combined.loc[group_indices, 'loan_approved']
    sensitive_group = sensitive_train_simplified.loc[group_indices, ['race_group', 'gender_group', 'ethnicity_group']]

    class_counts = y_group.value_counts()
    total_samples = len(y_group)
    
    # Simple logic based on provided code
    if total_samples < 100:
        # No resampling
        X_resampled_list.append(X_group)
        y_resampled_list.append(y_group)
        sensitive_resampled_list.append(sensitive_group)
    elif total_samples > 100000:
        # Under-sample
        rus = RandomUnderSampler(random_state=42)
        X_resampled_group, y_resampled_group = rus.fit_resample(X_group, y_group)
        X_resampled_list.append(pd.DataFrame(X_resampled_group, columns=feature_cols))
        y_resampled_list.append(pd.Series(y_resampled_group))
        sensitive_resampled_group = sensitive_group.sample(n=len(y_resampled_group), replace=True, random_state=42).reset_index(drop=True)
        sensitive_resampled_list.append(sensitive_resampled_group)
    elif len(class_counts) < 2 or class_counts.min() < 6:
        # Over-sample
        ros = RandomOverSampler(random_state=42)
        X_resampled_group, y_resampled_group = ros.fit_resample(X_group, y_group)
        X_resampled_list.append(pd.DataFrame(X_resampled_group, columns=feature_cols))
        y_resampled_list.append(pd.Series(y_resampled_group))
        sensitive_resampled_group = sensitive_group.sample(n=len(y_resampled_group), replace=True, random_state=42).reset_index(drop=True)
        sensitive_resampled_list.append(sensitive_resampled_group)
    else:
        # SMOTE
        sm = SMOTE(random_state=42)
        X_resampled_group, y_resampled_group = sm.fit_resample(X_group, y_group)
        X_resampled_list.append(pd.DataFrame(X_resampled_group, columns=feature_cols))
        y_resampled_list.append(pd.Series(y_resampled_group))
        sensitive_resampled_group = sensitive_group.sample(n=len(y_resampled_group), replace=True, random_state=42).reset_index(drop=True)
        sensitive_resampled_list.append(sensitive_resampled_group)

X_train_resampled = pd.concat(X_resampled_list, axis=0).reset_index(drop=True)
y_train_resampled = pd.concat(y_resampled_list, axis=0).reset_index(drop=True)
sensitive_train_resampled = pd.concat(sensitive_resampled_list, axis=0).reset_index(drop=True)

print("Resampling completed.")
print(f"Original dataset size: {X_train.shape[0]}")
print(f"Resampled dataset size: {X_train_resampled.shape[0]}")

# Encode race, gender, ethnicity for adversaries
def encode_series(series):
    label_encoder = LabelEncoder()
    int_encoded = label_encoder.fit_transform(series)
    onehot_encoder = OneHotEncoder(sparse_output=False)
    int_encoded = int_encoded.reshape(-1, 1)
    onehot_encoded = onehot_encoder.fit_transform(int_encoded)
    return onehot_encoded, label_encoder

race_onehot, _ = encode_series(sensitive_train_resampled['race_group'])
gender_onehot, _ = encode_series(sensitive_train_resampled['gender_group'])
ethnicity_onehot, _ = encode_series(sensitive_train_resampled['ethnicity_group'])

X_train_input = X_train_resampled.values.astype('float32')
y_train_output = y_train_resampled.values.astype('float32').reshape(-1,1)
race_train_output = race_onehot.astype('float32')
gender_train_output = gender_onehot.astype('float32')
ethnicity_train_output = ethnicity_onehot.astype('float32')

Resampling completed.
Original dataset size: 1281889
Resampled dataset size: 1059588


In [7]:
########################################
# Adversarial Model Setup
########################################

import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Input, Layer
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

class GradientReversalLayer(tf.keras.layers.Layer):
    def __init__(self):
        super(GradientReversalLayer, self).__init__()

    @tf.custom_gradient
    def call(self, x):
        def grad(dy):
            return -dy
        return x, grad

inputs = Input(shape=(16,))
x = Dense(64, activation='relu')(inputs)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
x = Dense(32, activation='relu')(x)
x = Dropout(0.5)(x)
main_output = Dense(1, activation='sigmoid', name='main_output')(x)

grl_layer = GradientReversalLayer()(x)

# Race adversary
race_adv = Dense(64, activation='relu')(grl_layer)
race_adv = BatchNormalization()(race_adv)
race_adv = Dropout(0.5)(race_adv)
race_adversary_output = Dense(race_train_output.shape[1], activation='softmax', name='race_adversary_output')(race_adv)

# Gender adversary
gender_adv = Dense(64, activation='relu')(grl_layer)
gender_adv = BatchNormalization()(gender_adv)
gender_adv = Dropout(0.5)(gender_adv)
gender_adversary_output = Dense(gender_train_output.shape[1], activation='softmax', name='gender_adversary_output')(gender_adv)

# Ethnicity adversary
ethnicity_adv = Dense(64, activation='relu')(grl_layer)
ethnicity_adv = BatchNormalization()(ethnicity_adv)
ethnicity_adv = Dropout(0.5)(ethnicity_adv)
ethnicity_adversary_output = Dense(ethnicity_train_output.shape[1], activation='softmax', name='ethnicity_adversary_output')(ethnicity_adv)

combined_model = Model(inputs=inputs, outputs=[main_output, race_adversary_output, gender_adversary_output, ethnicity_adversary_output])

combined_model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss={
        'main_output': 'binary_crossentropy',
        'race_adversary_output': 'categorical_crossentropy',
        'gender_adversary_output': 'categorical_crossentropy',
        'ethnicity_adversary_output': 'categorical_crossentropy'
    },
    loss_weights={
        'main_output': 1.0,
        'race_adversary_output': 0.5,
        'gender_adversary_output': 0.5,
        'ethnicity_adversary_output': 0.5
    },
    metrics={
        'main_output': 'accuracy',
        'race_adversary_output': 'accuracy',
        'gender_adversary_output': 'accuracy',
        'ethnicity_adversary_output': 'accuracy'
    }
)

print(combined_model.summary())

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history = combined_model.fit(
    X_train_input,
    {
        'main_output': y_train_output,
        'race_adversary_output': race_train_output,
        'gender_adversary_output': gender_train_output,
        'ethnicity_adversary_output': ethnicity_train_output
    },
    epochs=50,
    batch_size=64,
    validation_split=0.2,
    callbacks=[early_stopping],
    verbose=1
)

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 16)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_8 (Dense)     │ (None, 64)        │      1,088 │ input_layer_2[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 64)        │        256 │ dense_8[0][0]     │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_8 (Dropout) │ (None, 64)        │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_9 (Dense)     │ (None, 32)        │      2,080 │ dropout_8[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_9 (Dropout) │ (None, 32)        │          0 │ dense_9[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ gradient_reversal_… │ (None, 32)        │          0 │ dropout_9[0][0]   │
│ (GradientReversalL… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_10 (Dense)    │ (None, 64)        │      2,112 │ gradient_reversa… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_11 (Dense)    │ (None, 64)        │      2,112 │ gradient_reversa… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_12 (Dense)    │ (None, 64)        │      2,112 │ gradient_reversa… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 64)        │        256 │ dense_10[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 64)        │        256 │ dense_11[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 64)        │        256 │ dense_12[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_10          │ (None, 64)        │          0 │ batch_normalizat… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_11          │ (None, 64)        │          0 │ batch_normalizat… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_12          │ (None, 64)        │          0 │ batch_normalizat… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ main_output (Dense) │ (None, 1)         │         33 │ dropout_9[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ race_adversary_out… │ (None, 7)         │        455 │ dropout_10[0][0]  │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ gender_adversary_o… │ (None, 4)         │        260 │ dropout_11[0][0]  │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 11,601 (45.32 KB)

 Trainable params: 11,089 (43.32 KB)

 Non-trainable params: 512 (2.00 KB)

None
Epoch 1/50


2024-12-10 14:15:21.308640: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


13245/13245 ━━━━━━━━━━━━━━━━━━━━ 612s 46ms/step - ethnicity_adversary_output_accuracy: 0.3991 - gender_adversary_output_accuracy: 0.3102 - loss: 3.1976 - main_output_accuracy: 0.5533 - race_adversary_output_accuracy: 0.4098 - val_ethnicity_adversary_output_accuracy: 0.7994 - val_gender_adversary_output_accuracy: 0.2959 - val_loss: 3.6752 - val_main_output_accuracy: 0.6596 - val_race_adversary_output_accuracy: 0.0325
Epoch 2/50
13245/13245 ━━━━━━━━━━━━━━━━━━━━ 603s 46ms/step - ethnicity_adversary_output_accuracy: 0.5297 - gender_adversary_output_accuracy: 0.3693 - loss: 2.3513 - main_output_accuracy: 0.6422 - race_adversary_output_accuracy: 0.5715 - val_ethnicity_adversary_output_accuracy: 0.7994 - val_gender_adversary_output_accuracy: 0.3034 - val_loss: 3.6668 - val_main_output_accuracy: 0.8417 - val_race_adversary_output_accuracy: 0.0325
Epoch 3/50
13245/13245 ━━━━━━━━━━━━━━━━━━━━ 612s 46ms/step - ethnicity_adversary_output_accuracy: 0.5316 - gender_adversary_output_accuracy: 0.3733 -

In [8]:
########################################
# Evaluation on Test Set
########################################

test_predictions = combined_model.predict(X_test_scaled)
main_test_predictions_prob = test_predictions[0].reshape(-1)
main_test_predictions = (main_test_predictions_prob >= 0.5).astype(int)

def evaluate_model(y_true, y_pred, model_name):
    print(f"\n### {model_name} Performance ###")
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, zero_division=0)
    recall = recall_score(y_true, y_pred, zero_division=0)
    f1 = f1_score(y_true, y_pred, zero_division=0)
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall (TPR): {recall:.4f}")
    print(f"F1-Score: {f1:.4f}")
    print("Confusion Matrix:")
    cm = confusion_matrix(y_true, y_pred)
    cm_df = pd.DataFrame(cm, index=['Actual Negative','Actual Positive'], columns=['Predicted Negative','Predicted Positive'])
    display(cm_df)

evaluate_model(y_test.values, main_test_predictions, "Adversarial Neural Network with Resampled Data")

10015/10015 ━━━━━━━━━━━━━━━━━━━━ 16s 2ms/step

### Adversarial Neural Network with Resampled Data Performance ###
Accuracy: 0.8262
Precision: 0.9357
Recall (TPR): 0.7717
F1-Score: 0.8458
Confusion Matrix:


,Predicted Negative,Predicted Positive
Actual Negative,111987,10498
Actual Positive,45209,152779


In [13]:
########################################
# Prepare final test_results with SES, Race, Gender, Ethnicity
########################################

import pandas as pd

# Set pandas to display all rows and columns
pd.set_option('display.max_rows', None)  # Show all rows
pd.set_option('display.max_columns', None)  # Show all columns

test_results = pd.DataFrame(X_test_scaled, columns=feature_cols)
test_results['Actual'] = y_test.reset_index(drop=True)
test_results['Predicted'] = main_test_predictions

# Merge sensitive_test_simplified info
sensitive_test_simplified = sensitive_test_simplified.reset_index(drop=True)
test_results['Race'] = sensitive_test_simplified['race_group']
test_results['Gender'] = sensitive_test_simplified['gender_group']
test_results['Ethnicity'] = sensitive_test_simplified['ethnicity_group']

def categorize_ses(row):
    # Use tract_to_msa_income_percentage from scaled data:
    # We must revert scaling or use original df. Let's assume original logic:
    # If you want original data, re-merge from X_test. We can do this by inverse scaling or
    # since we know the threshold: <80 Low, <=120 Middle, else High
    # We'll trust that scaling didn't reorder data. We can just use original X_test.
    original_value = row['tract_to_msa_income_percentage']* (X_train['tract_to_msa_income_percentage'].max() - X_train['tract_to_msa_income_percentage'].min()) + X_train['tract_to_msa_income_percentage'].min()
    if original_value < 80:
        return 'Low'
    elif original_value <= 120:
        return 'Middle'
    else:
        return 'High'

# To apply SES categorization correctly, let's store original values before scaling:
# We'll just recategorize using original X_test directly:
X_test_reset = X_test.reset_index(drop=True)
test_results['original_tract_to_msa_income_percentage'] = X_test_reset['tract_to_msa_income_percentage'].values
test_results['SES'] = test_results['original_tract_to_msa_income_percentage'].apply(lambda v: 'Low' if v<80 else ('Middle' if v<=120 else 'High'))
test_results.drop('original_tract_to_msa_income_percentage', axis=1, inplace=True)

In [14]:
########################################
# Compute Fairness Metrics as Requested
########################################

# Functions to compute fairness metrics
def compute_approval_rate(df, group_cols):
    return df.groupby(group_cols)['Predicted'].mean()

def compute_statistical_parity(df, group_cols):
    # Same as approval rate
    return compute_approval_rate(df, group_cols)

def compute_predictive_parity(df, group_cols):
    metrics = {}
    for gvals, gdf in df.groupby(group_cols):
        precision = precision_score(gdf['Actual'], gdf['Predicted'], zero_division=0)
        metrics[gvals] = precision
    return pd.Series(metrics)

def compute_equal_opportunity(df, group_cols):
    metrics = {}
    for gvals, gdf in df.groupby(group_cols):
        recall = recall_score(gdf['Actual'], gdf['Predicted'], zero_division=0)
        metrics[gvals] = recall
    return pd.Series(metrics)

def compute_fpr_parity(df, group_cols):
    metrics = {}
    for gvals, gdf in df.groupby(group_cols):
        cm = confusion_matrix(gdf['Actual'], gdf['Predicted'], labels=[0,1])
        tn, fp, fn, tp = cm.ravel()
        fpr = fp / (fp + tn) if (fp+tn) > 0 else 0
        metrics[gvals] = fpr
    return pd.Series(metrics)

def compute_base_rate(df, group_cols):
    return df.groupby(group_cols)['Actual'].mean()

def compute_all_metrics(df, group_cols):
    approval = compute_approval_rate(df, group_cols)
    stat_parity = compute_statistical_parity(df, group_cols)
    pred_parity = compute_predictive_parity(df, group_cols)
    eq_opp = compute_equal_opportunity(df, group_cols)
    fpr = compute_fpr_parity(df, group_cols)
    base = compute_base_rate(df, group_cols)
    res = pd.DataFrame({
        'Approval Rate': approval,
        'Statistical Parity': stat_parity,
        'Predictive Parity': pred_parity,
        'Equal Opportunity': eq_opp,
        'FPR Parity': fpr,
        'Base Rate': base
    })
    return res

def format_table(df):
    for col in ['Approval Rate','Statistical Parity','Predictive Parity','Equal Opportunity','FPR Parity','Base Rate']:
        if col in df.columns:
            df[col] = df[col].apply(lambda x: f"{x:.2f}")
    return df

def split_multiindex(df, new_cols):
    df = df.reset_index()
    df.columns = list(new_cols) + list(df.columns[len(new_cols):])
    return df

In [15]:
########################################
# Full intersection: SES × Race × Gender × Ethnicity
########################################
full_metrics = compute_all_metrics(test_results, ['SES','Race','Gender','Ethnicity'])
full_table = split_multiindex(full_metrics, ['SES','Race','Gender','Ethnicity'])
full_table = format_table(full_table)

print("\n### Fairness Metrics by SES × Race × Gender × Ethnicity ###\n")
display(full_table)


### Fairness Metrics by SES × Race × Gender × Ethnicity ###



,SES,Race,Gender,Ethnicity,Approval Rate,Statistical Parity,Predictive Parity,Equal Opportunity,FPR Parity,Base Rate
0,Low,American Indian or Alaska Native,Female,Hispanic or Latino,0.34,0.34,0.91,0.73,0.05,0.42
1,Low,American Indian or Alaska Native,Female,Joint,0.08,0.08,1.00,0.50,0.00,0.15
2,Low,American Indian or Alaska Native,Female,Not Hispanic or Latino,0.41,0.41,0.92,0.68,0.07,0.56
3,Low,American Indian or Alaska Native,Female,Other,0.26,0.26,0.75,0.86,0.08,0.23
4,Low,American Indian or Alaska Native,Female,Unknown or Not Applicable,0.00,0.00,0.00,0.00,0.00,0.00
5,Low,American Indian or Alaska Native,Joint,Hispanic or Latino,0.38,0.38,0.93,0.70,0.05,0.50
6,Low,American Indian or Alaska Native,Joint,Joint,0.51,0.51,0.92,0.79,0.10,0.59
7,Low,American Indian or Alaska Native,Joint,Not Hispanic or Latino,0.43,0.43,0.92,0.72,0.07,0.55
8,Low,American Indian or Alaska Native,Joint,Other,0.50,0.50,1.00,0.75,0.00,0.67
9,Low,American Indian or Alaska Native,Joint,Unknown or Not Applicable,0.00,0.00,0.00,0.00,0.00,0.00


In [16]:
########################################
# Non-SES intersection: Race × Gender × Ethnicity
########################################
non_ses_metrics = compute_all_metrics(test_results, ['Race','Gender','Ethnicity'])
non_ses_table = split_multiindex(non_ses_metrics, ['Race','Gender','Ethnicity'])
non_ses_table = format_table(non_ses_table)

print("\n### Fairness Metrics by Race × Gender × Ethnicity ###\n")
display(non_ses_table)

print("All requested tables have been displayed.")


### Fairness Metrics by Race × Gender × Ethnicity ###



,Race,Gender,Ethnicity,Approval Rate,Statistical Parity,Predictive Parity,Equal Opportunity,FPR Parity,Base Rate
0,American Indian or Alaska Native,Female,Hispanic or Latino,0.34,0.34,0.91,0.73,0.05,0.42
1,American Indian or Alaska Native,Female,Joint,0.08,0.08,1.00,0.50,0.00,0.15
2,American Indian or Alaska Native,Female,Not Hispanic or Latino,0.41,0.41,0.92,0.68,0.07,0.56
3,American Indian or Alaska Native,Female,Other,0.26,0.26,0.75,0.86,0.08,0.23
4,American Indian or Alaska Native,Female,Unknown or Not Applicable,0.00,0.00,0.00,0.00,0.00,0.00
5,American Indian or Alaska Native,Joint,Hispanic or Latino,0.38,0.38,0.93,0.70,0.05,0.50
6,American Indian or Alaska Native,Joint,Joint,0.51,0.51,0.92,0.79,0.10,0.59
7,American Indian or Alaska Native,Joint,Not Hispanic or Latino,0.43,0.43,0.92,0.72,0.07,0.55
8,American Indian or Alaska Native,Joint,Other,0.50,0.50,1.00,0.75,0.00,0.67
9,American Indian or Alaska Native,Joint,Unknown or Not Applicable,0.00,0.00,0.00,0.00,0.00,0.00


All requested tables have been displayed.
